<a href="https://colab.research.google.com/github/oldairjsilva/IGTI/blob/main/Projeto_Aplicado_EDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Projeto Aplicado - EDD

O projeto aplicado tem como desafio coletar os dados financeiros de empresa listada na Bolsa de Valores e comparar com o Valor Intríseco calculado pela fórmula de Graham. 
O objetivo seria fazer uma pipeline de dados através do airflow para disponibilizar os dados dentro de um banco de dados para que um cientista de dados possa utilizar para fazer um dashboard a ser utilizado por um investidor para acompanhamento das empresas de sua carteira de ações.

#### Base de dados CVM
#### http://dados.cvm.gov.br

#### 1. Coletar dados financeiros das empresas na CVM
Acessar dados financeiros de empresas listadas da Bolsa de Valores na CVM e criar arquivos históricos

*   Baixar arquivos em formato zip do site da CVM dos anos de 2010 até 2022 por ano
*   Descompactar os arquivos
*   Unificar os arquivos por tipo e pelo conjunto de anos 2010 - 2022

#### 2. Acessar dados financeiros para gerar indicadores fundamentalistas e calcular a fórmula de Graham


In [25]:
import pandas as pd
import numpy as np

In [2]:
def filtrarUltimaDivulgacaoResultado(df):
    df = df[df['ORDEM_EXERC'] == "ÚLTIMO"]
    return df

In [3]:
def filtrarEmpresa(df, empresa):
    df = df[df['DENOM_CIA'].isin([empresa])]
    return df

In [4]:
def calcularLPA(df, codigoConta):
    df = df[df['CD_CONTA'] == codigoConta]
      # Seleciona as informações e renomeia a coluna - LPA
    df = df[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
    df.rename({'VL_CONTA':'LPA'}, axis=1, inplace=True)
    df['LPA'] = round(df['LPA'],2)
    return df

In [5]:
def calcularLucro(df, listaDescricaoConta):
    df = df[df['DS_CONTA'].isin(listaDescricaoConta)]
    # Seleciona as informações e renomeia a coluna - Lucro
    df = df[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
    df.rename({'VL_CONTA':'LUCRO'}, axis=1, inplace=True)
    df['LUCRO'] = df['LUCRO']*1000
    return df

In [22]:
def unificar_DRE_BPP(empresa, dre, bpp):
    df = pd.merge(dre, bpp, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')
    #df.to_csv(path_arquivos + f'/Silver/DRE_BPP_unificado - {empresa}.csv', index = False, encoding = 'utf=8', sep=';')
    #data_path = path_arquivos + f'/Silver/DRE_BPP_unificado - {empresa}.csv'
    return df

In [7]:
def calcularPatrimonioLiquido(df, listaDescricaoConta):
    df = df[df['DS_CONTA'].isin(listaDescricaoConta)]
    # Seleciona as informações e renomeia a coluna - Patrimonio Liquido
    df = df[['DT_REFER', 'DENOM_CIA', 'VL_CONTA']]
    df.rename({'VL_CONTA':'PATRIMONIO_LIQUIDO'}, axis=1, inplace=True)
    df['PATRIMONIO_LIQUIDO'] = df['PATRIMONIO_LIQUIDO']*1000        
    return df  

In [8]:
def tratarDemonstrativoResultadoExercicio(empresa):
    dre = pd.read_csv('/content/dfp_cia_aberta_DRE_con_2010-2022.csv', sep=';')
    dre = filtrarUltimaDivulgacaoResultado(dre)
    dre = filtrarEmpresa(dre, empresa)
    dreLPA = calcularLPA(dre, '3.99.01.01')
    dreLucro = calcularLucro(dre, ['Lucro/Prejuízo Consolidado do Período', 'Lucro ou Prejuízo Líquido Consolidado do Período'])        
    dre = pd.merge(dreLPA, dreLucro, on = ['DENOM_CIA', 'DT_REFER']).sort_values('DT_REFER')
    return dre

In [9]:
dreEmpresa = tratarDemonstrativoResultadoExercicio('BCO BRASIL S.A.')

In [10]:
dreEmpresa

,DT_REFER,DENOM_CIA,LPA,LUCRO
0,2010-12-31,BCO BRASIL S.A.,4.17,1.133034e+10
1,2011-12-31,BCO BRASIL S.A.,4.43,1.273691e+10
2,2012-12-31,BCO BRASIL S.A.,3.93,1.143820e+10
3,2013-12-31,BCO BRASIL S.A.,3.68,1.128883e+10
4,2014-12-31,BCO BRASIL S.A.,4.23,1.334350e+10
5,2015-12-31,BCO BRASIL S.A.,5.03,1.579804e+10
6,2016-12-31,BCO BRASIL S.A.,2.52,8.659577e+09
7,2017-12-31,BCO BRASIL S.A.,3.82,1.227530e+10
8,2018-12-31,BCO BRASIL S.A.,4.96,1.508610e+10
9,2019-12-31,BCO BRASIL S.A.,5.86,1.888832e+10


In [11]:
def tratarBalancoPatrimonial(empresa):
    bpp = pd.read_csv('/content/dfp_cia_aberta_BPP_con_2010-2022.csv', sep=';')
    bpp = filtrarUltimaDivulgacaoResultado(bpp)
    bpp = filtrarEmpresa(bpp, empresa)
    bpp = calcularPatrimonioLiquido(bpp, ['Patrimônio Líquido Consolidado'])
    #bpp.to_csv(path_arquivos + f'/Silver/balancoPatrimonial - {empresa}.csv', index = False, encoding = 'utf=8', sep=';')
    return bpp

In [12]:
balancoEmpresa = tratarBalancoPatrimonial('BCO BRASIL S.A.')

In [13]:
balancoEmpresa

,DT_REFER,DENOM_CIA,PATRIMONIO_LIQUIDO
39,2010-12-31,BCO BRASIL S.A.,5.441894e+10
78313,2011-12-31,BCO BRASIL S.A.,6.326922e+10
157029,2012-12-31,BCO BRASIL S.A.,6.989823e+10
234596,2013-12-31,BCO BRASIL S.A.,7.638200e+10
310618,2014-12-31,BCO BRASIL S.A.,8.544004e+10
385213,2015-12-31,BCO BRASIL S.A.,8.622999e+10
459383,2016-12-31,BCO BRASIL S.A.,9.007669e+10
533756,2017-12-31,BCO BRASIL S.A.,1.012384e+11
608170,2018-12-31,BCO BRASIL S.A.,1.045400e+11
682861,2019-12-31,BCO BRASIL S.A.,1.099714e+11


In [14]:
dre_bpp_unificado = unificar_DRE_BPP(dreEmpresa, balancoEmpresa)

In [15]:
dre_bpp_unificado

,DT_REFER,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO
0,2010-12-31,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10
1,2011-12-31,BCO BRASIL S.A.,4.43,1.273691e+10,6.326922e+10
2,2012-12-31,BCO BRASIL S.A.,3.93,1.143820e+10,6.989823e+10
3,2013-12-31,BCO BRASIL S.A.,3.68,1.128883e+10,7.638200e+10
4,2014-12-31,BCO BRASIL S.A.,4.23,1.334350e+10,8.544004e+10
5,2015-12-31,BCO BRASIL S.A.,5.03,1.579804e+10,8.622999e+10
6,2016-12-31,BCO BRASIL S.A.,2.52,8.659577e+09,9.007669e+10
7,2017-12-31,BCO BRASIL S.A.,3.82,1.227530e+10,1.012384e+11
8,2018-12-31,BCO BRASIL S.A.,4.96,1.508610e+10,1.045400e+11
9,2019-12-31,BCO BRASIL S.A.,5.86,1.888832e+10,1.099714e+11


In [16]:
def calcularQuantidadeAcoes(df):
    # Calcula a quantidade de ações a partir do lucro dividido por lucro por ação
    df['QTDE_ACOES'] = df['LUCRO']/df['LPA']
    df['QTDE_ACOES'] = df['QTDE_ACOES'].values.astype(np.int64) 
    return df 

In [17]:
def calcularValorPatrimonial(df):
    # Calcula o VPA(valor patrimonial por ação) dividindo o valor do patrimonio liquido pela quantidade de ações
    df['VPA'] = round(df['PATRIMONIO_LIQUIDO']/df['QTDE_ACOES'],2) 
    return df  

In [18]:
def calculaFormulaGraham(df_dre_bpp):
    #df = pd.read_csv(dre_bpp_path, sep=';')
    df = df_dre_bpp
    # Calcula o valor intrinseco da Formula de Graham
    df['VI_GRAHAM'] = round(np.sqrt(df['VPA'] * df['LPA'] * 22.5),2)
    return df 

In [19]:
def tratar_DRE_BPP_Unificados(empresa, dre_bpp):
    df = dre_bpp
    df = calcularQuantidadeAcoes(df)
    df = calcularValorPatrimonial(df)
    df = calculaFormulaGraham(df)
    #df.to_csv(path_arquivos + f'/Silver/DRE_BPP_unificado_tratado - {empresa}.csv', index = False, encoding = 'utf=8', sep=';')
    #data_path = path_arquivos + f'/Silver/DRE_BPP_unificado_tratado - {empresa}.csv'
    return df

In [76]:
def gerarIndicadoresFundamentalistas(empresa):
    bpp = tratarBalancoPatrimonial(empresa)
    dre = tratarDemonstrativoResultadoExercicio(empresa)
    dre_bpp = unificar_DRE_BPP(empresa, dre, bpp)
    dre_bpp_tratado = tratar_DRE_BPP_Unificados(empresa, dre_bpp)
    # Cria o index a partir da DT_REFER
    dre_bpp_tratado.index = pd.to_datetime(dre_bpp_tratado['DT_REFER'])
    # Remove a coluna DT_REFER depois de criar o indice
    del dre_bpp_tratado['DT_REFER']
    return dre_bpp_tratado

In [77]:
indicadoresEmpresa = gerarIndicadoresFundamentalistas('BCO BRASIL S.A.')

In [78]:
indicadoresEmpresa

,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
DT_REFER,,,,,,,
2010-12-31,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2717109112,20.03,43.35
2011-12-31,BCO BRASIL S.A.,4.43,1.273691e+10,6.326922e+10,2875149435,22.01,46.84
2012-12-31,BCO BRASIL S.A.,3.93,1.143820e+10,6.989823e+10,2910483460,24.02,46.09
2013-12-31,BCO BRASIL S.A.,3.68,1.128883e+10,7.638200e+10,3067617934,24.90,45.41
2014-12-31,BCO BRASIL S.A.,4.23,1.334350e+10,8.544004e+10,3154490780,27.09,50.78
2015-12-31,BCO BRASIL S.A.,5.03,1.579804e+10,8.622999e+10,3140763220,27.46,55.75
2016-12-31,BCO BRASIL S.A.,2.52,8.659577e+09,9.007669e+10,3436340079,26.21,38.55
2017-12-31,BCO BRASIL S.A.,3.82,1.227530e+10,1.012384e+11,3213430104,31.50,52.03
2018-12-31,BCO BRASIL S.A.,4.96,1.508610e+10,1.045400e+11,3041552620,34.37,61.93


In [29]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 11.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [32]:
import yfinance as yf

In [79]:
def baixarCotacoes(ativo):
    # Coletar cotações das ações no Yahoo Finance das empresas selecionadas
    df = yf.download(ativo, start='2011-01-01')[['Close', 'Adj Close']]

    # Renomear nome de colunas        
    df.rename({'Close':'PRECO_FECHAMENTO'}, axis=1, inplace=True)
    df.rename({'Adj Close':'PRECO_FECHAMENTO_AJUSTADO'}, axis=1, inplace=True)  

    # Ajustar valores para duas casas decimais
    df['PRECO_FECHAMENTO'] = round(df['PRECO_FECHAMENTO'],2)
    df['PRECO_FECHAMENTO_AJUSTADO'] = round(df['PRECO_FECHAMENTO_AJUSTADO'],2)
    return df  

In [80]:
cotacoes = baixarCotacoes('BBAS3.SA')

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-79-a97766127ed7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PRECO_FECHAMENTO'] = round(df['PRECO_FECHAMENTO'],2)
<ipython-input-79-a97766127ed7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [81]:
cotacoes

,PRECO_FECHAMENTO,PRECO_FECHAMENTO_AJUSTADO
Date,,
2011-01-03,31.40,14.24
2011-01-04,31.53,14.30
2011-01-05,31.98,14.51
2011-01-06,31.70,14.38
2011-01-07,30.95,14.04
...,...,...
2023-02-28,40.30,40.30
2023-03-01,39.00,39.00
2023-03-02,37.43,37.43


In [98]:
def gerarCotacoesComIndicadores(ativo, empresa):
    # Coletar cotações das ações no Yahoo Finance das empresas selecionadas
    cotacoes = baixarCotacoes(ativo)
   
    indicadoresEmpresa = gerarIndicadoresFundamentalistas(empresa)     

    # Unificar os Dados financeiros/indicadores/fórmula de Graham com as cotações
    #indicadores = cotacoes.join(indicadoresEmpresa, how='left')
    indicadores = cotacoes.join(indicadoresEmpresa, how='outer')

    # Preencher os indicadores/dados financeiros a partir da data de disponibilização para o ano seguinte
    indicadores.fillna(method='ffill', inplace = True)

    # Remover as linhas nulas referente aos dados de Fechamento(Close) e Fechamento ajustado(Adj Close)
    indicadores.dropna(inplace = True)

    # Retorna o data frame com as informações tratadas
    return indicadores  

In [99]:
cotacoesComIndicadores = gerarCotacoesComIndicadores('BBAS3.SA', 'BCO BRASIL S.A.')

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-79-a97766127ed7>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PRECO_FECHAMENTO'] = round(df['PRECO_FECHAMENTO'],2)
<ipython-input-79-a97766127ed7>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [100]:
cotacoesComIndicadores

,PRECO_FECHAMENTO,PRECO_FECHAMENTO_AJUSTADO,DENOM_CIA,LPA,LUCRO,PATRIMONIO_LIQUIDO,QTDE_ACOES,VPA,VI_GRAHAM
2011-01-03,31.40,14.24,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-04,31.53,14.30,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-05,31.98,14.51,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-06,31.70,14.38,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
2011-01-07,30.95,14.04,BCO BRASIL S.A.,4.17,1.133034e+10,5.441894e+10,2.717109e+09,20.03,43.35
...,...,...,...,...,...,...,...,...,...
2023-02-28,40.30,40.30,BCO BRASIL S.A.,6.43,1.972287e+10,1.461102e+11,3.067321e+09,47.63,83.01
2023-03-01,39.00,39.00,BCO BRASIL S.A.,6.43,1.972287e+10,1.461102e+11,3.067321e+09,47.63,83.01
2023-03-02,37.43,37.43,BCO BRASIL S.A.,6.43,1.972287e+10,1.461102e+11,3.067321e+09,47.63,83.01
2023-03-03,37.85,37.85,BCO BRASIL S.A.,6.43,1.972287e+10,1.461102e+11,3.067321e+09,47.63,83.01
